In [1]:

# Re-import dependencies (in case earlier import was skipped)
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from permutationsga.ga import (
    ConfigurableGA,

    RandomPermutationInitialization,
    crossover_pmx,
    TournamentSelection,
    FunctionBasedRecombinator,
    SequentialSelector,
    generate_sequential_indices,
    
    
)
from permutationsga.problem import IdenticalDecoder, InvPermDecoder, RandomKeysDecoder

from permutationsga.qap import QAP, read_qaplib
from mutation_initialization.frequency_based_initialization import FrequencyBasedInitialization
from our_other_crossover.linkage_based_crossover import LinkageBasedCrossover
from our_other_crossover.region_based_crossover import RegionBasedCrossover

In [2]:
# The bur* instances are those that we will be using - note that we are only using 26 keys in this case.
problem = QAP(*read_qaplib("./instances/qap/bur26a.dat"))

In [4]:
distance_matrix = problem.A
frequency_matrix = problem.B

if distance_matrix[0][0] == 53:
    type = 0
if distance_matrix[0][0] == 37:
    type = 1
else:
    Exception("The distance matrix is not as expected")

row_sum = np.sum(frequency_matrix, axis = 1)
col_sum = np.sum(frequency_matrix, axis = 0)
frequencies = np.add(row_sum, col_sum)
high_frequency = np.argsort(frequencies)[-8:]
print(f"type: {type}, high frequency characters: {high_frequency}")

type: 0, high frequency characters: [ 7 18  0 19 17  8 13  4]


In [5]:
row_sum = np.sum(distance_matrix, axis = 1)
col_sum = np.sum(distance_matrix, axis = 0)
distances = np.add(row_sum, col_sum)
better_region = np.argsort(distances)[:8]
print(better_region)

[18 17 16 12 10 11  9 13]


In [6]:
problem = IdenticalDecoder(problem) 

Recombine & Select (repeat a few times to see how things evolve!):

In [7]:
def pass_matrices_to_crossover():
    return frequency_matrix, distance_matrix

In [8]:
differences = []
p = 1e-2
seed = 42
population_size = 2**11
rng = np.random.default_rng(seed=seed + 1)
l = problem.get_length()


#crossover_fn = LinkageBasedCrossover().linkage_based_crossover
crossover_fn = RegionBasedCrossover().region_based_crossover
#initialization = FrequencyBasedInitialization(p, type, high_frequency)
initialization = RandomPermutationInitialization(l)
indices_gen = pass_matrices_to_crossover  # so that it gets passed to the crossover function
parent_selection = SequentialSelector()
recombinator = FunctionBasedRecombinator(
    indices_gen,
    crossover_fn,
    parent_selection,
    population_size * 2, # Note: double as we are including the previous population
    include_what="population"
)
selection = TournamentSelection()
ga = ConfigurableGA(
    seed, population_size, problem, initialization, recombinator, selection
)


In [ ]:
i=0
while True:
    ga.generation()
    differences.append(max(s.f for s in ga.population) - min(s.f for s in ga.population))
    if differences[-1] == 0:
        break
    i+=1 

In [ ]:
# plot the differences in plt.line
plt.plot(differences)

In [ ]:
print(len(differences))

In [ ]:
# Current best & worst
min(s.f for s in ga.population), max(s.f for s in ga.population)

In [ ]:
# print the genotype of the best
print(ga.population[np.argmin(s.f for s in ga.population)].e)

## Evaluating Linkage Based Crossover

In [9]:
from configs        import EA_Config, Exp_Config, rng, l
from run_experiment import run_experiments, round_to_multiple
from math           import ceil


p = 1e-2


setups = {
    "Baseline" :                                            EA_Config(crossover_fn=crossover_pmx, indices_gen=lambda: generate_sequential_indices(rng, l), initialization=RandomPermutationInitialization(l)),
    "Custom Initialisation" :                               EA_Config(crossover_fn=crossover_pmx, indices_gen=lambda: generate_sequential_indices(rng, l), initialization=FrequencyBasedInitialization(p, type, high_frequency, better_region)), 
    "Linkage Based Crossover Crossover" :                   EA_Config(crossover_fn=LinkageBasedCrossover().linkage_based_crossover, indices_gen=pass_matrices_to_crossover, initialization=RandomPermutationInitialization(l)), 
    "Custom Initialisation and Linkage Based Crossover" :   EA_Config(crossover_fn=LinkageBasedCrossover().linkage_based_crossover, indices_gen=pass_matrices_to_crossover, initialization=FrequencyBasedInitialization(p, type, high_frequency, better_region))
}

exp_cfg = Exp_Config("linkage_based_crossover", n_experiments=20)

# all_best_overtime, all_worst_overtime, all_iterations, all_total_times, all_iter_times = run_experiments(setups, exp_cfg)

# colors = ["orange", "blue", "green", "purple"]

# ncols = 2
# nrows = 2

# alltime_worst   = max([max(x) for x in all_worst_overtime])
# alltime_best    = min([min(x) for x in all_best_overtime])



In [10]:
all_fitnesses_overtime, all_iterations, all_total_times, all_iter_times = run_experiments(setups, exp_cfg)

Baseline
                                                                                                    
Custom Initialisation
                                                                                                    
Linkage Based Crossover Crossover
                                                                                                    
Custom Initialisation and Linkage Based Crossover
                                                                                                    


## Evaluating Region Based Crossover

In [11]:
from configs        import EA_Config, Exp_Config, rng, l
from run_experiment import run_experiments, round_to_multiple
from math       import ceil


p = 0.01 

setups = {
    "Baseline"    :                                         EA_Config(crossover_fn=crossover_pmx, indices_gen=lambda: generate_sequential_indices(rng, l), initialization=RandomPermutationInitialization(l)),
    "Custom Initialisation" :                               EA_Config(crossover_fn=crossover_pmx, indices_gen=lambda: generate_sequential_indices(rng, l), initialization=FrequencyBasedInitialization(p, type, high_frequency, better_region)), 
    "Region Based Crossover" :                              EA_Config(crossover_fn=RegionBasedCrossover().region_based_crossover, indices_gen=pass_matrices_to_crossover, initialization=RandomPermutationInitialization(l)), 
    "Custom Initialisation and Region Based Crossover" :    EA_Config(crossover_fn=RegionBasedCrossover().region_based_crossover, indices_gen=pass_matrices_to_crossover, initialization=FrequencyBasedInitialization(p, type, high_frequency, better_region))
}

exp_cfg = Exp_Config("region_based_crossover", n_experiments=20)

# all_best_overtime, all_worst_overtime, all_iterations, all_total_times, all_iter_times = run_experiments(setups, exp_cfg)

# colors = ["orange", "blue", "green", "purple"]

# ncols = 2
# nrows = 2

# alltime_worst   = max([max(x) for x in all_worst_overtime])
# alltime_best    = min([min(x) for x in all_best_overtime])



In [12]:
all_fitnesses_overtime, all_iterations, all_total_times, all_iter_times = run_experiments(setups, exp_cfg)

Baseline
                                                                                                    
Custom Initialisation
                                                                                                    
Region Based Crossover
                                                                                                    
Custom Initialisation and Region Based Crossover
                                                                                                    
